In [ ]:
# 高阶函数除了可以接受函数作为参数外，还可以把函数作为结果值返回。
# 我们来实现一个可变参数的求和。通常情况下，求和的函数是这样定义的：
def cacl_sum(*args):
    ax = 0
    print(args)
    for n in args:
        ax = ax + n
    return ax

# *args 的作用：*args 是一个可变参数，它会将传入的多个参数打包成一个元组。例如，调用 calc_sum(1, 2, 3) 时，args 的值会是 (1, 2, 3)。
# 传入列表时的行为：当你传入一个列表 [1, 3, 4] 时，args 的值会是 ([1, 3, 4],)，即一个包含一个列表的元组。
# 因此，for n in args 实际上只会遍历这个元组一次，n 的值是整个列表 [1, 3, 4]，而不是列表中的每个元素。
# 相当于是传入3个参数还是传入1个list参数
print(cacl_sum(1,3,4))

# **kwargs（通常写作 **kw 或 **kwargs）是一种用于接收任意数量的关键字参数的语法。
# 它允许函数在调用时接收任意数量的命名参数（即关键字参数），并将这些参数存储在一个字典中。
# **kwargs 的作用，**kwargs 可以接收任意数量的关键字参数。这些参数以键值对的形式传入函数，并被存储在一个字典中。
# 主要的参数是key=value形式，比如 name="Alice"变成了字典 {name:Alice}


(1, 3, 4)
8


In [9]:
# 但是，如果不需要立刻求和，而是在后面的代码中，根据需要再计算怎么办？可以不返回求和的结果，而是返回求和的函数：
def lazy_sum(*args):
    def sum():
        ax = 0
        for n in args:
            ax +=n
        return ax
    return sum

# 当我们调用lazy_sum()时，返回的并不是求和结果，而是求和函数：
f = lazy_sum(1,3,5,7,9)
print(f)

<function lazy_sum.<locals>.sum at 0x000001A490675120>


In [ ]:
# 调用函数f时，才真正计算求和的结果：
print(f())
# 个人理解是被返回了函数，要函数结果需要调用函数
# 在这个例子中，我们在函数lazy_sum中又定义了函数sum，并且，
# 内部函数sum可以引用外部函数lazy_sum的参数和局部变量，当lazy_sum返回函数sum时，
# 相关参数和变量都保存在返回的函数中，这种称为“闭包（Closure）”的程序结构拥有极大的威力。

25

In [ ]:
# 请再注意一点，当我们调用lazy_sum()时，每次调用都会返回一个新的函数，即使传入相同的参数：
f1 = lazy_sum(1,3,5,7,9)
f2 = lazy_sum(1,3,5,7,9)
print (f1 == f2) # 相当于两个一样的函数不同的地址所以不一样
# f1()和f2()的调用结果互不影响。
print(f1()==f2())# 但是计算出的数字结果是一样的

False
True


In [ ]:
# 闭包 Closure
# 注意到返回的函数在其定义内部引用了局部变量args，
# 所以，当一个函数返回了一个函数后，其内部的局部变量还被新函数引用，所以，闭包用起来简单，实现起来可不容易。
# 另一个需要注意的问题是，返回的函数并没有立刻执行，而是直到调用了f()才执行。我们来看一个例子：
def count():
    fs = []
    for i in range(1,4):
        def f():
            return i*i
        fs.append(f)
    return fs

f1 ,f2 , f3 = count()
# 在上面的例子中，每次循环，都创建了一个新的函数，然后，把创建的3个函数都返回了。
# 你可能认为调用f1()，f2()和f3()结果应该是1，4，9，但实际结果是：
print(f1())
print(f2())
print(f3())
# 全部都是9！原因就在于返回的函数引用了变量i，但它并非立刻执行。等到3个函数都返回时，它们所引用的变量i已经变成了3，因此最终结果为9。
# 在 Python 中，闭包引用的是变量的内存地址，而不是变量的当前值。
# 换句话说，闭包中的变量 i 是在函数被调用时才去查找其值，而不是在函数被定义时。
# 当 count() 函数执行完毕时，循环已经结束，变量 i 的最终值为 3。
# 之后f1()时才去查询i算i*i得到9
# 延迟绑定（late binding）


9
9
9


In [ ]:
# 返回闭包时牢记一点：返回函数不要引用任何循环变量，或者后续会发生变化的变量。
# 如果一定要引用循环变量怎么办？方法是再创建一个函数，
# 用该函数的参数绑定循环变量当前的值，无论该循环变量后续如何更改，已绑定到函数参数的值不变：
def count():
    def f(j):
        def g():
            return j*j
        return g
    fs = []
    for i in range(1,4):
        fs.append(f(i)) # f(i)立刻被执行，因此i的当前值传入f()
    return fs

f1 ,f2 , f3 = count()
print(f1())
print(f2())
print(f3())
# 缺点是代码较长，可利用lambda函数缩短代码。

1
4
9


In [13]:
# nonlocal
# 使用闭包，就是内层函数引用了外层函数的局部变量。如果只是读外层变量的值，我们会发现返回的闭包函数调用一切正常：
def inc():
    x = 0
    def fn():
        # 仅读取x的值
        return x+1
    return fn

f = inc()
print(f())
print(f())

1
1


In [14]:
# 但是，如果对外层变量赋值，由于Python解释器会把x当作函数fn()的局部变量，它会报错：
def inc():
    x = 0
    def fn():
        # nonlocal x
        x = x + 1
        return x
    return fn

f = inc()
print(f()) # 1
print(f()) # 2

UnboundLocalError: local variable 'x' referenced before assignment

In [ ]:
def inc():
    x = 0
    def fn():
        nonlocal x # 相当于声明了函数外变量
        x = x + 1
        return x
    return fn

f = inc()
print(f()) # 1
print(f()) # 2
# 原因是x作为局部变量并没有初始化，直接计算x+1是不行的。
# 但我们其实是想引用inc()函数内部的x，所以需要在fn()函数内部加一个nonlocal x的声明。
# 加上这个声明后，解释器把fn()的x看作外层函数的局部变量，它已经被初始化了，可以正确计算x+1。

# 使用闭包时，对外层变量赋值前，需要先使用nonlocal声明该变量不是当前函数的局部变量。

1
2


In [17]:
# 练习
# 利用闭包返回一个计数器函数，每次调用它返回递增整数：
def createCounter():
    x = 0
    def counter():
        nonlocal x
        x = x+1
        return x
    return counter

counterA = createCounter()
print(counterA(), counterA(), counterA(), counterA(), counterA()) # 1 2 3 4 5

counterB = createCounter()
if [counterB(), counterB(), counterB(), counterB()] == [1, 2, 3, 4]:
    print('测试通过!')
else:
    print('测试失败!')

1 2 3 4 5
测试通过!
